In [ ]:
# Below we have sections of short driver scripts that run different parts of the heatmap code
# Not all functions in the 5 functiond definition .py's are called yet,
# but the ones that are called so far compile below without error

#%load_ext autoreload
#%autoreload 2

In [ ]:
import importlib
import setup_nx # your own module, setup.nx.py
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
from operator import add
importlib.reload(setup_nx)
from setup_nx import *
from graphviz import Source, render

import datetime
import time

import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm


In [ ]:
# List functions in each helper .py
help(ff)
print("----------------------------------------------------------")
help(imp)
print("----------------------------------------------------------")
help(vis)
print("----------------------------------------------------------")
help(lzn)
print("----------------------------------------------------------")
help(hm)
print("----------------------------------------------------------")
help(ctrl)

In [ ]:
# [Essential] specify input feeder data

#-----------------------------------------------------------------
'specifying file paths'
# Enter the path/name of the model's excel file and import
    # All GridBright load files should be in the following folder
    #loadfolder = "/Users/jasperpakshong/Documents/Berkeley/ENERGISE/IEEE13/"
    #loadpath = loadfolder + "IEEE13testload_w_extreme_act.xlsx"

    # filepath = "IEEE13/"
    # modelpath = filepath + "001 phasor08_IEEE13_OPAL.xls"
    # loadfolder = "IEEE13/"
    # loadpath = loadfolder + "001_phasor08_IEEE13_norm03_HIL_7_1.xlsx"

    #filepath = "AL0001/"
    #modelpath = filepath + "AL0001_OPAL_working.xls"
    #loadfolder = "AL0001/"
    #loadpath = loadfolder + "AL0001_tvload_afternoon1h.csv"
    
    #filepath = "13NF_balanced/"
    #modelpath = filepath + "016 GB_IEEE13_balance_all_ver2.xls"
    #loadfolder = "13NF_balanced/"
    #loadpath = loadfolder + "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondWise_norm03.csv"

filepath = "13NF_balanced/"
modelpath = filepath + "impedMod_IEEE13 unbal copy.xlsx" #"016 GB_IEEE13_balance_all_ver2.xls"
loadfolder = "13NF_balanced/"
loadfile = "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondwise_norm03.csv"
loadpath = loadfolder + loadfile

#==========================================================================================================

'specifying file name'
#file_name = string specifying name of dot file created when make_graph() is called
file_name = '13NF_test.dot'

#==========================================================================================================

'list actuator locations and performance node'
# all_act_locs = list of node names as strings where actuators will be placed
all_act_locs = ['bus_651','bus_611','bus_633']
perf_nodes = ['bus_651','bus_611','bus_633']

#==========================================================================================================

'Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_650'
timesteps = 1

'DO NOT NEED TO EDIT THIS CELL BEFORE RUNNING'

ts = time.time()
print()
print(datetime.datetime.fromtimestamp(ts))

plot = 0 #turn plot on/off

depths = {}
leaves = []

In [ ]:
# --------- [ESSENTIAL] create feeder obj ----------
fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix

print(depths) # should be populated

count = 0 # print list of buses in network
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
Source.from_file(file_name)
#^ need this to plot feeder

In [ ]:
# ------ [Optional] run impedance-related functions --------
slack_bus = None
for bus_name, depth in depths.items():
    if depth == 0:
        slack_bus = bus_name
        break
print(depths)
# modify node names when change feeders
imp.plot_histogram_RX_ratios(fin_feeder,leaves,slack_bus,depths,leaves_only = True)
print('Z between buses:')
print(imp.get_total_impedance_between_two_buses(fin_feeder, 'bus_632', 'bus_684',depths))
print('\nZ to substation:')
print(imp.get_total_impedance_from_substation(fin_feeder, 'bus_632',depths))
print('\nX/R ratio of path to substation:')
print(imp.get_RX_ratio_tosubst(fin_feeder, 'bus_632',depths))
# should check on how to format the printing do that it's to like 2 decimal places

In [ ]:
# ----- run detLznRange ---------
#node_1 = list(fin_feeder.network.successors(substation_name))
#z12 = imp.get_total_impedance_from_substation(fin_feeder, node_1[0],depths) # 3 phase
#lzn.detLznRange(fin_feeder, Vbase_ll, Sbase, z12, all_act_locs) # usually called by computeFeas
#^ calls computePQsweep, makePVcurve, makeQVcurve, computeLznItvl

In [ ]:
# ----- run computePQsweep ---------

loadfolder = "13NF_balanced/"
load_data = '001_phasor08_IEEE13_time_PVpen50_cloud_minutewise (1).csv'
lzn.computePQsweep_timesteps(fin_feeder, loadfolder+load_data)

In [ ]:
# ----- Evaluate detControlMatExistence ---------
# BOGUS DATA just to run through it:

n=1 # Temp
A = np.identity(6*n)
sz=(6*n,6*n)
B=np.ones(sz) # TEMP: replace with call to createRXmatrices_3ph
indicMat=np.identity(6*n) # TEMP: replace with actual config you want to evaluate
#act_locs=
#perf_nodes=
#feeder=

MYfeas,MYfeasFs,MYnumfeas,MYnumTried,MYnumact=ctrl.detControlMatExistence(A, B, indicMat)
#MYfeas,MYfeasFs,MYnumfeas,MYnumTried,MYnumact=detControlMatExistence(feeder, act_locs, perf_nodes,A, B, indicMat):

print("--------------------- Outputs ----------------------")
print("Myfeas=",MYfeas)
#print("MyfeasFs=\n",MYfeasFs)
print("Mynumfeas=",MYnumfeas)
print("MynumTried=",MYnumTried)
print("Mynumact=",MYnumact)

print('computeFeas_v1 returns:')
hm.computeFeas_v1(fin_feeder, perf_nodes, A, B, indicMat,substation_name,depths)


In [ ]:
# ----- Create Fig 1 ---------
# start with just binary heat map across all nodes
# based on heat map colors, maybe try: branches color-coded and marked feasible (green outline) or infeasible (red outline) 
# define feeder, define node_index_map
# define act locs = {}
# perf node = colocated with candidate act node <-- Brittany can we do this?

# colocated?
all_act_locs = []
perf_nodes = []
# empty act locs asks heatMapProcess to create heatmap for empty network, i.e. feas of placing 1 co-located actuator at each loc
feas_configs, heatMapNames=hm.find_good_colocated(fin_feeder, node_index_map,substation_name,depths, file_name)
print('----------- Feas configs are: ---------')
print(feas_configs)
Source.from_file(heatMapNames[0]) # display graph in notebook


In [ ]:
# ----- Create Figure 2 ---------
ratios = vis.phaseCouplingPerNode(fin_feeder,depths)
vis.createColorMap(fin_feeder, ratios)
Source.from_file('colorMap') # display graph in notebook

In [ ]:
# ------ Define a bunhc of feas configs -------
all_act_locs = ['bus_632','bus_632','bus_645','bus_652']
perf_nodes = ['bus_671','bus_675','bus_632','bus_652']

In [ ]:
def computeFParamSpace_v2(feeder, act_locs, perf_nodes,R,X,depths,node_index_Map):
    # Compute (Fp,Fq) ranges as a func of impedance paths between act nodes, perf nodes, and substation
    gamma=np.array([0.08, 0.35]) # scaling, according to units of Q-V and P-delta loops
    c=np.array([1, 1])
    avgSens_dvdq,avgSens_ddeldp= (np.empty((0,1)) for i in range(2))
    
    for act in act_locs: # for each (perf,act) pair
        print(node_index_Map)
        act_idx=node_index_Map[act] # index of act node
        print('act idx=',act_idx)
        perf_idx=act_idx #index of act node
        perf=perf_nodes[0] # temp
        # indexing  a[start:stop] --> items start through stop-1
        print('evaluating act at ',act,', perf at ',perf)

        Zgood=np.empty((3,3))
        Zgood=(R[act_idx*3:(act_idx*3+3),perf_idx*3:(perf_idx*3+3)])/2+(X[act_idx*3:(act_idx*3+3),perf_idx*3:(perf_idx*3+3)])/2*1j # 3x3 matrix        
        Z_toSubst=imp.get_total_impedance_from_substation(feeder,act,depths)
        Z_actperf=imp.get_total_impedance_between_two_buses(feeder,act,perf,depths)
        print('Z act perf=',Z_actperf)

        #Zgood and Z_tosubts should have same value, but they dont...
        print('Zgood=\n',Zgood) # divide by 2 because R and X construction doubles the impedance paths
        print('Z to subst=',Z_toSubst)

        Zbad1=np.subtract(Z_toSubst,Zgood) # >=0
        Zbad2=np.subtract(Z_actperf,Zbad1) # >=0
        #Zbad2=imp.get_total_impedance_between_two_buses(feeder,act,perf,depths) - Zbad1
        print('Zbad1=',np.around(Zbad1,2))
        print('Zbad2=',np.around(Zbad2,2))
        # deratings should range 0 to 1 for each 3x3 elements
        der1=np.ones(3)-np.divide(c[0]*Zbad1,Zgood+Zbad1) # derating for actuator not colocated
        der2=np.ones(3)-np.divide(c[1]*Zbad2,Zgood+Zbad2) # derating for perf node not on samepath to substation as act
        print('der1=',der1)
        print('der2=',der2)
        mainPath=Zgood
        sensEst_dvdq=gamma[0]*np.multiply(np.multiply(der1,der2),mainPath)
        sensEst_ddeldp=gamma[1]*np.multiply(np.multiply(der1,der2),mainPath)

        avg_dvdq=sensEst_dvdq.mean() # converts 3x3 to scalar complex
        avg_ddeldp=sensEst_ddeldp.mean()
        
        avgSens_dvdq=np.append(avgSens_dvdq,avg_dvdq)
        avgSens_ddeldp=np.append(avgSens_ddeldp,avg_ddeldp)
        print('avgdvdq=',avgSens_dvdq)
        print('avgddeldp=',avgSens_ddeldp)
        
    # avgSens_dvdq is list of scalar complex vals
    Fq_ub=np.amax(np.absolute(avgSens_dvdq)) # take max abs value of perf-act pair sensitivities
    Fp_ub=np.amax(np.absolute(avgSens_ddeldp))
   # Fq_lb=1
   # Fp_lb=2
    return Fq_ub,Fp_ub




# ----- Run computeFParamSpace  and save R and X matrices as csv ---------

#importlib.reload(ctrl)
all_act_locs = list(['bus_675'])
perf_nodes = list(['bus_675'])
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

Fq_ub,Fp_ub=computeFParamSpace_v2(fin_feeder, all_act_locs, perf_nodes,R,X,depths,node_index_map)
# this function prints Zgood (from R and X matrices) and Z_toSubst, where Z_toSubst is correct but doesnt match Zgood. Investigate
print('Fq_ub=',Fq_ub)
print('Fp_ub=',Fp_ub)

# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
#np.savetxt('Rmat_13NF.csv', R, delimiter=',')
#np.savetxt('Xmat_13NF.csv', X, delimiter=',')

In [ ]:
# -------- Solve 3-ph fwd-bwd sweep for toy example ----------
# values have been validated wrt matlab translation
R12=np.identity(3) # per unit
X12=3*np.identity(3)
V1=1*np.ones((3,1))
V2=np.array([[0.97, 0.98, 0.985]]).transpose()
del2=np.array([[2, -1, -7]]).transpose() # degrees
V2cplx=np.multiply(V2,np.cos(del2*np.pi/180))+np.multiply(V2,np.sin(del2*np.pi/180)*1j)
I=np.dot(np.linalg.inv(R12+1j*X12),V1-V2cplx) # I=YV, 3x1 = (3x3)(3x1) matrix mult
S2=np.multiply(V2cplx,np.conj(I)); # 3x1=(3x1)(3x1) elementwise!
P2=np.real(S2)
Q2=np.imag(S2);

# inputs: 3x3 matrices and 3x1 vectors
V2_3ph,del2_3ph=lzn.solveFwdBwdSweep_2bus_3ph(R12, X12, V1, P2, Q2)
# outputs: 3x1 vector for mag (Vpu), 3x1 vector for phase angle (degrees)
print('V2_3ph=',V2_3ph)
print('del2_3ph=',del2_3ph)

print('--------------------------------------------------')
R12=0.37878788*np.array([[ 0.3465,  0.156,  0.158,], # not pu
       [ 0.156,  0.3375,  0.1535,],
       [ 0.158,  0.1535,  0.3414]])
X12=0.37878788*np.array([[ 1.0179,  0.5017,  0.4236,],
       [ 0.5017,  1.0478,  0.3849,],
       [ 0.4236,  0.3849,  1.0348]])
V1=Vbase*np.ones((3,1)) # 2401
P2=100000*np.ones((3,1)) # watts, to be congruent with volts and ohms
Q2=20000*np.ones((3,1))
V2_3ph,del2_3ph=lzn.solveFwdBwdSweep_2bus_3ph(R12, X12, V1, P2, Q2)
print('V2_3ph=',V2_3ph)
print('del2_3ph=',del2_3ph)

In [ ]:
# -------Create Fig 5 ----------
# Desc: Two diagrams for 2 complex config1 and 2, labeled with act/perf node locations 

# Define Complex Config 3 (infeas)
all_act_locs = ['bus_675','bus_611','bus_652','bus_645']
perf_nodes = ['bus_671','bus_684','bus_684','bus_632']

# normally file_name='13NF_test.dot'
vis.markActuatorConfig(all_act_locs, fin_feeder, 'complex_config_1')

# Define Complex Config 4 (feas)
all_act_locs = ['bus_611','bus_675','bus_645','bus_646']
perf_nodes = ['bus_671','bus_671','bus_632','bus_646']

vis.markActuatorConfig(all_act_locs, fin_feeder, 'complex_config_2')

# BUG: second call to markActCOnfig appends actuator nodes to previous graph

In [ ]:
# ------- Run evalConfig on complex config 4 (feas) ----------
all_act_locs = ['bus_611','bus_675','bus_645','bus_646']
perf_nodes = ['bus_671','bus_671','bus_632','bus_646']
feas, maxError=hm.eval_config(fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name)
    # ^ calls setupStateSPace,updateStateSpace,computeFeas_v1
    # our R and X matrices are different between MATLAB and python...

In [ ]:
# -------Create Fig 4 ----------
# ----------- Demo of RHP on FEAS config --------------

# binary heat map that shows good locations to place a new actuator when
# a) there are 0 existing actuators, b) there is 1 existing actuator, and c) there are two existing actuators.
# actuator list = complex config 1, i.e. at each step we choose one of the good places and buildup to complex config 1

#complex config 4 (feas)
all_act_locs = ['bus_611','bus_675','bus_645','bus_646']
perf_nodes = ['bus_671','bus_671','bus_632','bus_646']
lst_feas_configs, lzn_error_run_sum,heatMapNames=hm.runHeatMapProcess(fin_feeder, all_act_locs, perf_nodes, node_index_map,substation_name,depths,file_name)
#see sections below to view graph results

In [ ]:
Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
Source.from_file(heatMapNames[1]) # display graph in notebook

In [ ]:
Source.from_file(heatMapNames[2]) # display graph in notebook

In [ ]:
Source.from_file(heatMapNames[3]) # display graph in notebook

In [ ]:
# ----------- Demo of RHP on INFEAS config --------------
# Let's see where feas is lost! With complex config 3 (infeas)
all_act_locs = ['bus_611','bus_652','bus_645','bus_675',]
perf_nodes = ['bus_684','bus_684','bus_632','bus_671']
lst_feas_configs, lzn_error_run_sum,heatMapNames=hm.runHeatMapProcess(fin_feeder, all_act_locs, perf_nodes, node_index_map,substation_name,depths,file_name)
#see sections below to view graph results

In [ ]:
Source.from_file(heatMapNames[0]) # display graph in notebook
# we place 611-->684

In [ ]:
Source.from_file(heatMapNames[1]) # display graph in notebook
# we place 652-->684

In [ ]:
Source.from_file(heatMapNames[2]) # display graph in notebook
# we place 645-->632


In [ ]:
Source.from_file(heatMapNames[3]) # display graph in notebook
# we are looking to track 671, but stupidly choose 675 as act node even though heat map shows red
# we place 675-->671, resulting in infeas config
